<h1> 1. Preparation:

<h2> 1.1 Modules:

In [41]:
from ultralytics import YOLO
import cv2
import cvzone
import math
import os
import numpy as np

<h2> 1.2 Support Functions:

In [33]:
# Support function to check if two line segments intersect with each other

# Return true if ABC are in counterclock wise order (if slope AC is larger than slope AB then yes)
def ccw(Ax, Ay, Bx, By, Cx, Cy):
    return (Cy-Ay) * (Bx-Ax) > (By-Ay) * (Cx-Ax)

# Return true if line segments AB and CD intersect
def intersect(Ax, Ay, Bx, By, Cx, Cy, Dx, Dy):
    return ccw(Ax, Ay, Cx, Cy, Dx, Dy) != ccw(Bx, By, Cx, Cy, Dx, Dy) and ccw(Ax, Ay, Bx, By, Cx, Cy) != ccw(Ax, Ay, Bx, By, Dx, Dy)

In [53]:
# Support function to display the coordinates of the points clicked on the image
def click_event(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f'({x},{y})')
      
        # Put coordinates as text on the image
        cv2.putText(img, f'({x},{y})',(x,y),
        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
      
        # Draw point on the image
        cv2.circle(img, (x, y), 3, (0, 255, 255), -1)

<h2> 1.3 Perspective Transformation Preparation:

In [57]:
# Read the first frame from video for coordinate matching
cap = cv2.VideoCapture(r".\videos\testVideo3.mp4")
flag, frame = cap.read()
cv2.imwrite("firstframe.jpg", frame)
cap.release()
cv2.destroyAllWindows()

In [59]:
# Read the image of the half court
img = cv2.imread('firstframe.jpg')

# Create a window
cv2.namedWindow('Point Coordinates')

# Bind the callback function to window
cv2.setMouseCallback('Point Coordinates', click_event)

# Display the image
while True:
    cv2.imshow('Point Coordinates',img)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
cv2.destroyAllWindows()

(254,520)
(384,606)
(637,454)
(934,497)


In [60]:
# Read the image of the half court
img = cv2.imread('halfCourtBirdsEye.jpg')

# Create a window
cv2.namedWindow('Point Coordinates')

# Bind the callback function to window
cv2.setMouseCallback('Point Coordinates', click_event)

# Display the image
while True:
    cv2.imshow('Point Coordinates',img)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
cv2.destroyAllWindows()

(90,198)
(179,203)
(52,25)
(180,23)


In [66]:
# Record the corresponding 4 corners for perspective transformation
pts1 = np.array([[254, 520], [384, 606], [637, 454], [934, 497]], np.float32)
pts2 = np.array([[90, 198], [179, 203], [52, 25], [180,23]], np.float32)

In [105]:
M = cv2.getPerspectiveTransform(pts1, pts2)

# Add another support function for perspective transformation
def perspectiveTransform(x, y):
    pt = np.array([[x,y]],  dtype = 'float32')
    pt = np.array([pt])
    return (cv2.perspectiveTransform(pt, M)[0][0])

In [137]:
print(perspectiveTransform(146, 518))

[     69.581      220.77]


<h1> 2. Data Processing:

In [140]:
# Set color to red in RGB
color = (255, 255, 255) 

# Read video from path
cap = cv2.VideoCapture(r".\videos\testVideo5.mp4")

# Get the parameters of the input video
width = int(cap.get(3))
height = int(cap.get(4))
fps = cap.get(5)

# Set the parameters for the output video 
out = cv2.VideoWriter('testVideo3Make3.mp4', -1, fps, (int(width), int(height)))

# Read the half court in birds eye view image
img = cv2.imread('halfCourtBirdsEye.jpg')
# Get half court img size
img_height, img_width, img_layer = img.shape

# Load the YOLO model created from Main.py
model_rim = YOLO(r".\runs\detect\train\weights\best.pt")
model_ball = YOLO(r".\runs\detect\train2\weights\best.pt")

# Load all the class names
rim_classNames = model_rim.names
ball_classNames = model_ball.names

# Initialize variables with default value
topOfHoop_x = -1000 # x-coord of the center at the top of the basketball hoop bounding box
topOfHoop_y = -1000 # y-coord of the top of the hoop box
bottomOfHoop_y = -1000 # y-coord of the bottom of the hoop box
widthOfHoop = 0 # width of the hoop box
floorOfBall_x = 0 # x-coord of the center at the lowest floor of ball bounding box
floorOfBall_y = 0 # y-coord of the lowest floor of ball box
heightOfBall = 0 # height of the ball box
leftUpperHoop_x = 0 
leftUpperHoop_y = 0
rightUpperHoop_x = 0
rightUpperHoop_y = 0
centerOfBall_x = 0
centerOfBall_y = 0
shooting_x = 0
shooting_y = 0
h = 20 # closeness range of ball and hoop (in pixel)
nearTheTopOfHoop = False 
ball_detected = False
frameCountDown = 0
makeCountDown = 0
num_attempt = 0
num_make = 0
score_positions = []
fail_positions = []

while True:
    
    # Get the next frame from the video
    flag, frame = cap.read()
    print(flag)
    
    # Check if all frames are extracted
    if not flag:
        break
    
    frame[height - img_height: height, width - img_width: width] = img
    
    # Run the YOLO model for rim detection and classification
    results = model_rim(frame, stream = True, conf = 0.7)
    
    previous_centerOfBall_x = centerOfBall_x
    previous_centerOfBall_y = centerOfBall_y
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            if rim_classNames[int(box.cls)] == 'rim':
                # Get the coordinates of the bounding box (left-upper corner and right-lower corner)
                x1, y1, x2, y2 = box.xyxy[0]
                # Draw the bounding box to the frame
                frame = cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2) 
                # Write the identified class name at the left-lower corner of the box
                frame = cv2.putText(frame, rim_classNames[int(box.cls)], (int(x1), int(y2)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
                # Record the coord information of the hoop
                topOfHoop_x = int((x2 + x1) / 2)
                leftUpperHoop_x = x1
                leftUpperHoop_y = y1
                rightUpperHoop_x = x2
                rightUpperHoop_y = y1
                widthOfHoop = int(x2 - x1)
                topOfHoop_y = int(y1)
                bottomOfHoop_y = int(y2)
                
            if rim_classNames[int(box.cls)] == 'people':
                # Get the coordinates of the bounding box (left-upper corner and right-lower corner)
                x1, y1, x2, y2 = box.xyxy[0]
                # Draw the bounding box to the frame
                frame = cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2) 
                # Write the identified class name at the left-lower corner of the box
                frame = cv2.putText(frame, rim_classNames[int(box.cls)], (int(x1), int(y2)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                
                # Record the coord information of the person shooting
                centerOfPeople_x = int((x2 + x1) / 2)
                floorOfPeople_y = int(y2)
                
    # Run the YOLO model for ball detection and classification
    results = model_ball(frame, stream = True, conf = 0.3)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            if ball_classNames[int(box.cls)] == 'basketball':
                ball_detected = True
                # Get the coordinates of the bounding box (left-upper corner and right-lower corner)
                x1, y1, x2, y2 = box.xyxy[0]
                # Draw the bounding box to the frame
                frame = cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2) 
                # Write the identified class name at the left-lower corner of the box
                frame = cv2.putText(frame, ball_classNames[int(box.cls)], (int(x1), int(y2)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                
                # Record the coord information of the hoop
                floorOfBall_x = int((x2 + x1) / 2)
                floorOfBall_y = int(y2)
                centerOfBall_x = floorOfBall_x
                centerOfBall_y = floorOfBall_y - heightOfBall_y / 2
                heightOfBall_y = int(y2 - y1)
    
    # Check if the ball is near the upper region of the hoop
    ballNearHoopX = (floorOfBall_x > (topOfHoop_x - widthOfHoop / 2)) & (floorOfBall_x < (topOfHoop_x + widthOfHoop / 2))
    ballNearHoopY = (floorOfBall_y > (topOfHoop_y - 100)) & (floorOfBall_y < (topOfHoop_y + 20))
    
    # Check if the person is in shooting form by tracking the position of the ball and the person
    if ((centerOfBall_x - 100) < centerOfPeople_x) & ((centerOfPeople_x + 100) < width):
        shooting_x = centerOfPeople_x
        shooting_y = floorOfPeople_y
    
    # Check if there is a make by connecting the center of the ball in the current frame and in the previous frame where the ball is detected
    if (ball_detected):
        if (intersect(previous_centerOfBall_x, previous_centerOfBall_y, centerOfBall_x, centerOfBall_y, leftUpperHoop_x, leftUpperHoop_y, rightUpperHoop_x, rightUpperHoop_y)):
            # if the connecting segment intersects the horizontal hoop, there is a make
            if (makeCountDown == 0):
                adjustedPeople_x = int(width - img_width + perspectiveTransform(shooting_x, shooting_y)[0])
                adjustedPeople_y = int(height - img_height + perspectiveTransform(shooting_x, shooting_y)[1])
                
                # write to the log file when a make takes place
                file = open('log.txt', 'a')
                file.write('score with position in ' + str(adjustedPeople_x) + ' and ' + str(adjustedPeople_y) + '\n')
                file.close()
                
                score_positions.append([adjustedPeople_x, adjustedPeople_y])
                num_make += 1
                makeCountDown = 60
                if not nearTheTopOfHoop:
                    num_attempt += 1
                    frameCountDown = 60
                else:
                    fail_positions = fail_positions[:-1]
                
    # Check if there is an attempt
    if (nearTheTopOfHoop):
        if (frameCountDown == 0):
            num_attempt += 1
            adjustedPeople_x = int(width - img_width + perspectiveTransform(shooting_x, shooting_y)[0])
            adjustedPeople_y = int(height - img_height + perspectiveTransform(shooting_x, shooting_y)[1])
            fail_positions.append([adjustedPeople_x, adjustedPeople_y])
            frameCountDown = 60
    
    if (ballNearHoopX & ballNearHoopY):
        nearTheTopOfHoop = True
    else:
        nearTheTopOfHoop = False
        #print('very close')
        #frame = cv2.putText(frame, 'very close', (600,600), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 3)

    if (frameCountDown != 0):
        frameCountDown -= 1
            
    if (makeCountDown != 0):
            makeCountDown -= 1
            
    for i in range(0, len(score_positions)):
        frame = cv2.circle(frame, (score_positions[i][0], score_positions[i][1]), 10, (0, 255, 0), 1)
    for i in range(0, len(fail_positions)):
        frame = cv2.circle(frame, (fail_positions[i][0], fail_positions[i][1]), 10, (0, 0, 255), 1)

    frame = cv2.putText(frame, 'Make:' + str(num_make), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 3)
    frame = cv2.putText(frame, 'Attempt:' + str(num_attempt), (50,110), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 3)
        
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

True

0: 384x640 1 rim, 8.0ms
Speed: 2.5ms preprocess, 8.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 rim, 7.7ms
Speed: 2.0ms preprocess, 7.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.5ms
Speed: 2.0ms preprocess, 7.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 17.1ms
Speed: 2.0ms preprocess, 17.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.5ms
Speed: 2.0ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 0.0m

True

0: 384x640 1 people, 1 rim, 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.5ms
Speed: 2.0ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 8.0ms
Speed: 1.5ms preprocess, 8.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference

0: 384x640 1 people, 1 rim, 8.3ms
Speed: 1.0ms preprocess, 8.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 11.7ms
Speed: 1.0ms preprocess, 11.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 2.

True

0: 384x640 1 basketball, 1 people, 1 rim, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 6.5ms
Speed: 1.0ms preprocess, 6.5ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 6.5ms
Speed: 1.2ms preprocess, 6.5ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 8.5ms
Speed: 1.1ms preprocess, 8.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.5ms
Speed: 1.0ms preprocess, 7

Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 2 rims, 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 2 rims, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 8.0ms
Speed: 2.4ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 2 rims, 7.7ms
Speed: 1.0ms preprocess, 7.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.1ms
Speed: 2.0ms preprocess, 8.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 2 rims, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.5ms postprocess per 

...


0: 384x640 (no detections), 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.5ms
Speed: 2.0ms preprocess, 7.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.8ms
Speed: 1.0ms preprocess, 7.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 8.7ms
Speed: 1.0ms preprocess, 8.7

Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 2 rims, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 2 rims, 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.0ms
Speed: 2.5ms preprocess, 8.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 2 rims, 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 2 rims, 8.0ms
Speed: 1.5ms preprocess, 8.0ms inference, 1.0ms postprocess


0: 384x640 1 basketball, 7.7ms
Speed: 1.0ms preprocess, 7.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 2 peoples, 1 rim, 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 8.4ms
Speed: 1.0ms preprocess, 8.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 6.9ms
Speed: 2.6ms preprocess, 6.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.5ms
Speed: 2.0ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.0ms
Speed: 2.5ms preprocess, 7.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inferenc


0: 384x640 1 basketball, 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 7.5ms
Speed: 1.3ms preprocess, 7.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.7ms
Speed: 1.0ms preprocess, 7.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 15.5ms
Speed: 1.0ms preprocess, 15.5ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 15.5ms
Speed: 1.0ms preprocess, 15.5ms infer


0: 384x640 1 basketball, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 15.5ms
Speed: 1.0ms preprocess, 15.5ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 16.4ms
Speed: 1.0ms preprocess, 16.4ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 15.5ms
Speed: 2.0ms preprocess, 15.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 people, 1 rim, 8.0ms
Speed: 1.5ms preprocess, 8.0ms inf

<h1> 3. Testings

In [8]:
img = cv2.imread('testImage1.jpg')
model = YOLO(r"C:\Users\grandrio\BasketballScoringYolo\runs\detect\train\weights\best.pt")
classNames = model.names
results = model(img, stream = True)

In [9]:
color = (255, 0, 0) 
for r in results:
    boxes = r.boxes
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        img = cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2) 
        print(classNames[int(box.cls)])
        img = cv2.putText(img, classNames[int(box.cls)], (int(x1), int(y2)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

cv2.imwrite('test.jpg', img)  


0: 384x640 1 Basketball Hoop, 1 Basketball, 21.1ms
Basketball
Basketball Hoop
Speed: 3.1ms preprocess, 21.1ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)


True

In [7]:
cv2.imshow('image', img)
cv2.waitKey(0)

-1